# In Class 4 - Dario Placencio.

Consider data set HealthData.txt. We build a neural network to predict the number of ER visit, i.e. 'ernum'.

In [55]:
# Import the required libraries
import pandas as pd

In [56]:
# Read the HealData.txt
data = pd.read_table("HealthData.txt", sep=' ')

In [57]:
# Describe the data
data.head()

,year,age,female,race,married,edu,income,msa,region,limitation,chronic,smoke,industry,occupation,uninsured,erexp,ernum
0,2010,28,0,1,1,14,67000,1,1,0,3,0,6,7,0,0,0
1,2010,25,1,1,1,14,67000,1,1,0,1,0,-1,-1,0,0,0
2,2010,51,0,6,1,17,91420,1,4,0,0,0,10,2,0,0,0
3,2010,53,1,4,1,17,91420,1,4,0,0,0,10,2,0,0,0
4,2010,33,0,1,1,12,19500,1,3,0,0,1,11,7,1,0,0


In [58]:
# Check for missing values
print(data.isnull().sum())

year          0
age           0
female        0
race          0
married       0
edu           0
income        0
msa           0
region        0
limitation    0
chronic       0
smoke         0
industry      0
occupation    0
uninsured     0
erexp         0
ernum         0
dtype: int64


### Split data training and test sets, using observations in year 2010 as training and observations in year 2011 as test. Show the number of observations in each set.

In [59]:
# Filter the data for year 2010 and year 2011
train_data = data[data['year'] == 2010]
test_data = data[data['year'] == 2011]

train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

# Print the number of observations in each set
print("Number of observations in the training set: ", len(train_data))
print("Number of observations in the test set: ", len(test_data))

Number of observations in the training set:  17847
Number of observations in the test set:  19134


### Fit a Poisson neural network using three inputs: age, income (in log scale), and industry. Start with tuning parameters: (1) embedding dimension = 1; (2) number of hidden layers = 1; (3) number of neurons = 2. Calculate test data MSE.

In [12]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

In [60]:
from sklearn.preprocessing import StandardScaler

# Filter the data for the three input variables and year 2010 only
train_data = data[data['year'] == 2010][['age', 'industry', 'income', 'ernum']]
test_data = data[data['year'] == 2011][['age', 'industry', 'income', 'ernum']]

# Log transform the income variable
train_data['income'] = np.log(train_data['income'])
test_data['income'] = np.log(test_data['income'])

# Standardize the input variables except ernum
scaler = StandardScaler()
train_data[['age', 'industry', 'income']] = scaler.fit_transform(train_data[['age', 'industry', 'income']])
test_data[['age', 'industry', 'income']] = scaler.transform(test_data[['age', 'industry', 'income']])

# Split the input and output variables
X_train = train_data.drop('ernum', axis=1)
y_train = train_data['ernum']
X_test = test_data.drop('ernum', axis=1)
y_test = test_data['ernum']

In [62]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error

# Define the Poisson neural network model
model = Sequential()
model.add(Embedding(input_dim=1, output_dim=1, input_length=3))
model.add(Dense(units=2, activation='relu'))
model.add(Dense(units=1, activation='exponential'))

# Compile the model
model.compile(loss='poisson', optimizer='adam')

# Fit the model
early_stop = EarlyStopping(monitor='val_loss', patience=5)
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stop])

# Predict on the test data and calculate the mean squared error
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print('Test data MSE:', mse)

Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node 'sequential_3/embedding_8/embedding_lookup' defined at (most recent call last):
    File "c:\Users\Placenciohid\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\Placenciohid\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\Placenciohid\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\Placenciohid\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "c:\Users\Placenciohid\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Users\Placenciohid\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\Placenciohid\anaconda3\lib\asyncio


    From 2, change embedding dimension to 2, does this improve test data MSE?

    From 2, change number of neurons to 5, does this improve test MSE?

    From 2, change number of hidden layers to 2, does this improve test MSE?
